In [3]:
from datasets import load_from_disk, load_dataset
from datasets import load_dataset, load_from_disk
import sys
import json
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor

from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.spice.spice import Spice
from pycocoevalcap.cider.cider import Cider
mplug = load_dataset('alexshengzhili/mPLUG-owl', split = '1_percent_as_validation')


Found cached dataset parquet (/home/ubuntu/.cache/huggingface/datasets/alexshengzhili___parquet/alexshengzhili--mPLUG-owl-803e207c32dd7d84/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


In [5]:
import json

# load this file /home/ubuntu/LLaVA-Graph/evaluation/llava-7b-v0/llava_answers_3k.jsonl
# into a list of dictionaries
json_file = '/home/ubuntu/LLaVA/data/3k_response.jsonl'
with open('/home/ubuntu/LLaVA-Graph/evaluation/llava-13b-v0/llava30b_3k.jsonl', 'r') as f:
    llava_answers_3k = [json.loads(line) for line in f]

In [6]:
llava_list  = [item['text'] for item in llava_answers_3k]

In [7]:
len(llava_list), len(mplug)

(3002, 3002)

In [8]:
for i in range(3000):
    assert llava_answers_3k[i]['prompt'] == mplug[i]['q_a_pairs'][0][0]

In [9]:
llava = mplug.remove_columns(['response_mPLUG-owl'])
llava = llava.add_column('llava_response', llava_list)

In [11]:
def evaluate_captions(gt_captions, pred_captions, print_scores=True):
    evaluators = [Bleu(4), Rouge(), Cider(), Spice()]
    
    results = {}
    for evaluator in evaluators:
        score, _ = evaluator.compute_score(gt_captions, pred_captions)
        if isinstance(evaluator, Bleu):
            score = score[-1]  # Use the BLEU-4 score
        results[evaluator.method()] = score
    
    if print_scores:
        print("----- Evaluation Scores -----")
        for metric, score in results.items():
            print(f"{metric}: {score}")
    
    return results

def captions_list_to_dict(captions_list):
    captions_dict = {}
    for idx, caption in enumerate(captions_list):
        captions_dict[idx] = [caption]
    return captions_dict

gt_mplug = [example[0][1] for example in llava['q_a_pairs']]
pred_mplug = [example['llava_response'] for example in llava]
evaluate_captions(captions_list_to_dict(gt_mplug),
                    captions_list_to_dict(pred_mplug))

{'testlen': 260736, 'reflen': 189848, 'guess': [260736, 257734, 254732, 251730], 'correct': [68791, 23152, 11868, 7529]}
ratio: 1.3733934516033808


Parsing reference captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.4 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [0.9 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.5 sec].
Loading classif

SPICE evaluation took: 5.701 min
----- Evaluation Scores -----
Bleu: 0.07580728525727966
Rouge: 0.2338612863697905
CIDEr: 0.08635428996862568
SPICE: 0.16187201130237872


{'Bleu': 0.07580728525727966,
 'Rouge': 0.2338612863697905,
 'CIDEr': 0.08635428996862568,
 'SPICE': 0.16187201130237872}

In [16]:
llava.push_to_hub('alexshengzhili/llava-eval', type = 'dataset')


Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:01<00:00,  1.68s/it]
